# EddyPro API

This notebook gives an outline for working with the API.

* First step is to define your input primary parameters.  You need to give a siteID, everything else is *optional*.

    * kwargs (dict of all optional settings):\
        * sourceDir: if not provided, assumes data are already in Outputs folder
        * dateRange (if not provided, will run all files in the current year)
        * fileType (defaults to .ghg) must specify otherwise for 
        * metadataTemplate (only required for .dat files)



In [9]:
# def makeUserConfig()


# Example run for BB on one day in 2024
siteID = 'BB'
kwargs = {
    'dateRange':['2024-04-01','2024-04-02'],
    'sourceDir':"Y:/BB/",
    'reset':True
}

# # # Example run for All BBS data from the Datadump Folder 20240516
# siteID = 'BBS'
# kwargs = {
#     'dateRange':['2023-06-01','2024-05-31'],
#     'sourceDir':"X:/BBS/EC_Station/2024/20240516/",
#     'searchTag':"BBS.FLUX",
#     'timeShift':30,
#     'fileType':'dat',
#     'metaDataTemplate':'Y:/BBS/TOA5_BBS.FLUX_2023_06_14_1500.metadata',
# }


# # Cospectral correction options
# hf_meth = {
#     'Moncrieff et al. (1997)':'1',
#     'Horst (1997)':'2',
#     'Ibrom et al. (2007)':'3',
#     'Fratini et al. (2012)':'4', # Recommended for closed path systems, when sufficient data (>1 month) are available
#     'Massman (2000, 2001)':'5'
# }

# if siteID == 'BBS':
#     userDefinedEddyProSettings = {
#         'Project':{
#             'hf_meth':'1'
#             },
#         'RawProcess_Settings':{
#             'v_offset':0.08,
#             'u_offset':-0.04
#             },
#         }
#     kwargs['userDefinedEddyProSettings'] = userDefinedEddyProSettings

# if 'dateRange' not in kwargs:
#     kwargs['dateRange'] = None


# Check the configurations

In [11]:
import eddyProAPI
import importlib
importlib.reload(eddyProAPI)
kwargs['queryBiometDatabase'] = True
eP = eddyProAPI.eddyProAPI(siteID,**kwargs)
eP.config['BiometUser']

{'dbase_metadata': {'timestamp': {'name': 'clean_tv', 'dtype': 'float64', 'precision': 8, 'base': 719529, 'base_unit': 'D', 'resolution': '30min'}, 'traces': {'dtype': 'float32', 'precision': 4}}, 'stage': {'Second': 'Clean/SecondStage', 'Third': 'Clean/ThirdStage', 'Third_Default': 'Clean/ThirdStage_Default_Ustar', 'Third_Advanced': 'Clean/ThirdStage_Full_Ustar_Uncertainty', 'Met': 'Met', 'Flux': 'Flux', 'Manual': 'Met/Manual'}, 'rootDir': {'Database': 'C:/Database/', 'Outputs': 'C:/temp/', 'Datadump': 'X:/'}, 'tasks': {'biometData': {'formatting': {'units_in_header': True, 'na_value': -9999, 'time_vectors': {'timestamp': {'output_name': 'TIMESTAMP_1', 'fmt': '%Y-%m-%d %H%M', 'units': 'yyyy-mm-dd HHMM'}}}, 'traces': {'LW_IN_1_1_1': {'units': 'W+1m-2', 'output_name': 'LWIN_1_1_1'}, 'PA_1_1_1': {'units': 'kPa', 'output_name': 'Pa_1_1_1'}, 'PPFD_IN_1_1_1': {'units': 'umol+1m-2s-1', 'output_name': 'PPFD_1_1_1'}, 'RH_1_1_1': {'units': '%', 'output_name': 'RH_1_1_1'}, 'SW_IN_1_1_1': {'units

{'Biomet.net': 'C:/Biomet.net/', 'Database': 'C:/Database/'}

In [8]:
# import yaml
# with open('config_files/user_path_definitions.yml') as yml:
#     config = yaml.safe_load(yml)
# config
import os
os.path.isfile('config_files/user_path_definitions.yml')

True

## Run PreProcessing

In [3]:
pre = eddyProAPI.preProcessing(siteID,**kwargs)

{'dbase_metadata': {'timestamp': {'name': 'clean_tv', 'dtype': 'float64', 'precision': 8, 'base': 719529, 'base_unit': 'D', 'resolution': '30min'}, 'traces': {'dtype': 'float32', 'precision': 4}}, 'stage': {'Second': 'Clean/SecondStage', 'Third': 'Clean/ThirdStage', 'Third_Default': 'Clean/ThirdStage_Default_Ustar', 'Third_Advanced': 'Clean/ThirdStage_Full_Ustar_Uncertainty', 'Met': 'Met', 'Flux': 'Flux', 'Manual': 'Met/Manual'}, 'rootDir': {'Database': 'C:/Database/', 'Outputs': 'C:/temp/', 'Datadump': 'X:/'}, 'tasks': {'biometData': {'formatting': {'units_in_header': True, 'na_value': -9999, 'time_vectors': {'timestamp': {'output_name': 'TIMESTAMP_1', 'fmt': '%Y-%m-%d %H%M', 'units': 'yyyy-mm-dd HHMM'}}}, 'traces': {'LW_IN_1_1_1': {'units': 'W+1m-2', 'output_name': 'LWIN_1_1_1'}, 'PA_1_1_1': {'units': 'kPa', 'output_name': 'Pa_1_1_1'}, 'PPFD_IN_1_1_1': {'units': 'umol+1m-2s-1', 'output_name': 'PPFD_1_1_1'}, 'RH_1_1_1': {'units': '%', 'output_name': 'RH_1_1_1'}, 'SW_IN_1_1_1': {'units

# Run EddyPro in Parallel over date range

In [4]:

import eddyProAPI
import importlib
importlib.reload(eddyProAPI)

eP = eddyProAPI.runEP(siteID,**kwargs)

{'dbase_metadata': {'timestamp': {'name': 'clean_tv', 'dtype': 'float64', 'precision': 8, 'base': 719529, 'base_unit': 'D', 'resolution': '30min'}, 'traces': {'dtype': 'float32', 'precision': 4}}, 'stage': {'Second': 'Clean/SecondStage', 'Third': 'Clean/ThirdStage', 'Third_Default': 'Clean/ThirdStage_Default_Ustar', 'Third_Advanced': 'Clean/ThirdStage_Full_Ustar_Uncertainty', 'Met': 'Met', 'Flux': 'Flux', 'Manual': 'Met/Manual'}, 'rootDir': {'Database': 'C:/Database/', 'Outputs': 'C:/temp/', 'Datadump': 'X:/'}, 'tasks': {'biometData': {'formatting': {'units_in_header': True, 'na_value': -9999, 'time_vectors': {'timestamp': {'output_name': 'TIMESTAMP_1', 'fmt': '%Y-%m-%d %H%M', 'units': 'yyyy-mm-dd HHMM'}}}, 'traces': {'LW_IN_1_1_1': {'units': 'W+1m-2', 'output_name': 'LWIN_1_1_1'}, 'PA_1_1_1': {'units': 'kPa', 'output_name': 'Pa_1_1_1'}, 'PPFD_IN_1_1_1': {'units': 'umol+1m-2s-1', 'output_name': 'PPFD_1_1_1'}, 'RH_1_1_1': {'units': '%', 'output_name': 'RH_1_1_1'}, 'SW_IN_1_1_1': {'units